# Imports

Various useful builtins:

In [1]:
from typing import *
import functools
import pathlib

External packages:

In [2]:
import pandas as pd
import numpy as np

Our main workhorse, `allensdk`:

In [3]:
import allensdk
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import EcephysSession

Local modules:

In [4]:
from constraints import *

# Global Variables
We have a bunch of variables which we will realistically set only once
and never again.  Because of that, and to save up on namespace, we
will write these in uppercase and type them as `Final`.

## Introspection & debugging
First are the two debugging variables, `DEBUG` and `EXAMPLE`.  
`DEBUG` should control behavior checks.  
`EXAMPLE` should control examples which are not necessary for the project.

In [5]:
DEBUG: Final[bool] = False
EXAMPLE: Final[bool] = True

and we need a simple way to output something only when `EXAMPLE` is
`True`, so we define the function `example`

In [6]:
def example(x):
  if EXAMPLE:
    return x  

## Paths
Then we have our data directory, `DATA_DIR`, which contains the data
we cache from `allensdk`.

In [7]:
DATA_DIR: Final[pathlib.Path] = pathlib.Path('./data/')

and we need to ensure that this directory exists

In [8]:
DATA_DIR.mkdir(parents=True, exist_ok=True)

Furthermore, we have the path to the manifest file, again relative to
the project root, which is by default the `manifest.json` file located
directly in `DATA_DIR`.

In [9]:
MANIFEST_PATH: Final[pathlib.Path] = DATA_DIR / 'manifest.json'

## Cache & sessions
Next is our `allensdk` cache and the sessions in it.

First define the type aliases `Cache` and `Session` for easier
typing in functions.

In [10]:
Cache: TypeAlias = EcephysProjectCache
Session: TypeAlias = EcephysSession

Set the cache's timeout, `CACHE_TIMEOUT`, which is measured in
seconds:

In [11]:
CACHE_TIMEOUT: Final[int] = 30*60 # 30 minutes

Define the global cache, `CACHE`,

In [12]:
CACHE: Final[Cache] = EcephysProjectCache.from_warehouse(manifest=MANIFEST_PATH, timeout=CACHE_TIMEOUT)

Load the table of sessions for simple access.

In [13]:
SESSIONS_TABLE: Final = CACHE.get_session_table()
example(SESSIONS_TABLE.head())

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


Get the complete list of session ids.

In [14]:
SESSION_IDS: Final[Sequence[int]] = SESSIONS_TABLE.index
example(SESSION_IDS)

Int64Index([715093703, 719161530, 721123822, 732592105, 737581020, 739448407,
            742951821, 743475441, 744228101, 746083955, 750332458, 750749662,
            751348571, 754312389, 754829445, 755434585, 756029989, 757216464,
            757970808, 758798717, 759883607, 760345702, 760693773, 761418226,
            762120172, 762602078, 763673393, 766640955, 767871931, 768515987,
            771160300, 771990200, 773418906, 774875821, 778240327, 778998620,
            779839471, 781842082, 786091066, 787025148, 789848216, 791319847,
            793224716, 794812542, 797828357, 798911424, 799864342, 816200189,
            819186360, 819701982, 821695405, 829720705, 831882777, 835479236,
            839068429, 839557629, 840012044, 847657808],
           dtype='int64', name='id')

We typically want to work on just one session, `CURRENT_SESSION`.
That can be specified here for convenience using `CURRENT_SESSION_ID`.

Functions strive to accept a `session` parameter where meaningful and
default to `CURRENT_SESSION`.

In [15]:
%%capture
CURRENT_SESSION_ID: Final[int] = 715093703
assert CURRENT_SESSION_ID in SESSION_IDS
CURRENT_SESSION: Final[Session] = CACHE.get_session_data(CURRENT_SESSION_ID)
example(CURRENT_SESSION.metadata)

# Accessing filtered data
The `Session` object exposes a lot of useful functions and properties
to us.  In order to free us from having to always type the session
(which will overwhelmingly be `CURRENT_SESSION`) and to simultaneously
allow us to filter our data without having to go through much trouble,
we will define simple wrappers around those functions.

Typically, these functions accept a `session` parameter which defaults
to `CURRENT_SESSION`, and a bunch of keyword arguments which filter
the resulting dataset.

Each argument `foo=bar`, unless stated otherwise, filters the `foo`
column of the resulting dataset to values matching `bar`.  See the
commentary at the start of `constraints.py` for a full description of
what values `bar` can take.

In [16]:
def get_sessions(**kwargs):
    """Return a table of the matching sessions.

    The following filters are meaningful:
    - published_at                (time)
    - specimen_id                 (integer, key)
    - session_type                ('brain_observatory_1.1' or 'functional_connectivity')
    - age_in_days                 (float)
    - sex                         ('M' or 'F')
    - full_genotype               (string)
    - unit_count                  (integer)
    - channel_count               (integer)
    - probe_count                 (integer)
    - ecephys_structure_acronyms  (list of strings)

    If an argument `__total__=False` is passed, additional filters may
    be provided with no effect on the result.

    """
    return filter_df(SESSIONS_TABLE, FIELD(**kwargs))

def get_session_ids(**kwargs):
    """Return the matching session ids.

    See `get_sessions` for the list of meaningful filters.
    
    """
    return get_sessions(**kwargs).index

example(get_sessions(sex='M', unit_count=RANGE(650, None)))

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
744228101,2019-10-03T00:00:00Z,719817805,brain_observatory_1.1,122.0,M,wt/wt,659,2226,6,"[Eth, TH, LP, POL, APN, DG, CA1, VIS, nan, CA3..."
750332458,2019-10-03T00:00:00Z,726141251,brain_observatory_1.1,91.0,M,wt/wt,902,2216,6,"[grey, VISrl, nan, VISal, IntG, IGL, LGd, CA3,..."
750749662,2019-10-03T00:00:00Z,726162197,brain_observatory_1.1,92.0,M,wt/wt,761,2223,6,"[LP, DG, CA1, VISp, nan, LGd, CA3, VISrl, VPM,..."
754829445,2019-10-03T00:00:00Z,726298253,brain_observatory_1.1,141.0,M,wt/wt,832,1851,5,"[PoT, LP, LGd, CA3, DG, CA1, VISp, nan, VPM, C..."
755434585,2019-10-03T00:00:00Z,730760270,brain_observatory_1.1,100.0,M,Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,650,2220,6,"[grey, VISrl, nan, MGv, MGd, TH, LGd, CA3, DG,..."


In [18]:
def get_units(ecephys_structure_acronym = None,
              unit_ids = None,
              session: Session = CURRENT_SESSION,
              **kwargs):
    """Return a `Session.units` dataframe of the matching units in `session`.

    The `unit_ids` argument narrows the stimulus presentations
    considered to those whose id it contains.
    
    The following filters are meaningful:
    - waveform_PT_ratio                      (float)
    - waveform_amplitude                     (float)
    - amplitude_cutoff                       (float)
    - cluster_id                             (integer, key)
    - cumulative_drift                       (float)
    - d_prime                                (float or null)
    - firing_rate                            (float)
    - isi_violations                         (float)
    - isolation_distance                     (float or null)
    - L_ratio                                (float or null)
    - local_index                            (integer)
    - max_drift                              (float)
    - nn_hit_rate                            (float or null)
    - nn_miss_rate                           (float or null)
    - peak_channel_id                        (integer, key)
    - presence_ratio                         (float)
    - waveform_recovery_slope                (float or null)
    - waveform_repolarization_slope          (float)
    - silhouette_score                       (float or null)
    - snr                                    (float)
    - waveform_spread                        (float)
    - waveform_velocity_above                (float or null)
    - waveform_velocity_below                (float or null)
    - waveform_duration                      (float)
    - filtering                              (string)
    - probe_channel_number                   (integer)
    - probe_horizontal_position              (integer)
    - probe_id                               (integer)
    - probe_vertical_position                (integer)
    - structure_acronym                      (string)
    - ecephys_structure_id                   (float, key)
    - ecephys_structure_acronym              (string)
    - anterior_posterior_ccf_coordinate      (float or null)
    - dorsal_ventral_ccf_coordinate          (float or null)
    - left_right_ccf_coordinate              (float or null)
    - probe_description                      (string, probeA..F)
    - location                               (object)
    - probe_sampling_rate                    (float)
    - probe_lfp_sampling_rate                (float)
    - probe_has_lfp_data                     (bool)

    If an argument `__total__=False` is passed, additional filters may
    be provided with no effect on the result.

    """
    if ecephys_structure_acronym is not None:
        kwargs['ecephys_structure_acronym'] = ecephys_structure_acronym

    units = session.units
    if unit_ids is not None:
        units = units.loc[unit_ids]

    return filter_df(units, FIELD(**kwargs))
    
def get_unit_ids(ecephys_structure_acronym = None,
                 unit_ids = None,
                 session: Session = CURRENT_SESSION,
                 **kwargs):
    """Return the matching unit ids in `session`.
    
    See `get_units` for the list of meaningful filters.

    """
    return get_units(ecephys_structure_acronym = ecephys_structure_acronym,
                     unit_ids = unit_ids, session = session, **kwargs).index

example(get_units(isi_violations = RANGE(None, 0.7),
                  structure_acronym = 'VISam').head())

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
950911880,0.149041,154.688625,0.028614,194,90.20,3.253001,10.106237,0.024401,55.535226,0.039318,...,394.0,VISam,7455.0,1140.0,7585.0,probeA,See electrode locations,29999.954846,1249.998119,True
950911873,0.212998,62.672025,0.015878,193,194.33,5.005439,2.923758,0.030688,66.411449,0.001101,...,394.0,VISam,7455.0,1140.0,7585.0,probeA,See electrode locations,29999.954846,1249.998119,True
950911932,0.195936,51.526605,0.007200,199,220.66,3.925777,5.176069,0.024479,62.670387,0.004846,...,394.0,VISam,7444.0,1104.0,7592.0,probeA,See electrode locations,29999.954846,1249.998119,True
950911986,0.175944,61.300785,0.003619,204,192.31,4.187314,3.917164,0.025645,59.651327,0.006886,...,394.0,VISam,7422.0,1031.0,7604.0,probeA,See electrode locations,29999.954846,1249.998119,True
950912018,0.071366,47.595600,0.092690,207,87.73,4.379658,16.813593,0.023354,100.593158,0.005868,...,394.0,VISam,7427.0,1050.0,7601.0,probeA,See electrode locations,29999.954846,1249.998119,True


In [21]:
def get_stimulus_presentations(stimulus_name = None,
                               stimulus_presentation_ids = None,
                               stimulus_condition_id = None,
                               session: Session = CURRENT_SESSION,
                               **kwargs):
    """Return the Sessions.stimulus_presentations dataframe of `session`.

    The `stimulus_presentation_ids` argument narrows the stimulus
    presentations considered to those whose id it contains.
    
    The following filters are meaningful:
    - stimulus_block           (float or null, key) 
    - start_time               (float)
    - stop_time                (float)
    - contrast                 (float or null) 
    - spatial_frequency        (float, string, or null) 
    - frame                    (float or null) 
    - stimulus_name            (string) 
    - x_position               (float or null)
    - y_position               (float or null) 
    - orientation              (float or null) 
    - temporal_frequency       (float or null) 
    - size                     (object) 
    - color                    (-1.0, 1.0, or null) 
    - phase                    (object) 
    - duration                 (float)
    - stimulus_condition_id    (integer, key)

    If an argument `__total__=False` is passed, additional filters may
    be provided with no effect on the result.

    """
    if stimulus_name is not None:
        kwargs['stimulus_name'] = stimulus_name
    if stimulus_condition_id is not None:
        kwargs['stimulus_condition_id'] = stimulus_condition_id

    stimulus_presentations = session.stimulus_presentations
    if stimulus_presentation_ids is not None:
        stimulus_presentations = stimulus_presentations.loc[stimulus_presentation_ids]
    return filter_df(stimulus_presentations, FIELD(**kwargs))

def get_stimulus_presentation_ids(stimulus_name = None,
                                  stimulus_presentation_ids = None,
                                  stimulus_condition_id = None,
                                  session: Session = CURRENT_SESSION,
                                  **kwargs):
    """Return the matching stimulus presentation ids in `session`.
        
    See `get_stimulus_presentations` for a list of meaningful filters.
    
    """
    return get_stimulus_presentations(stimulus_name = stimulus_name,
                                      stimulus_presentation_ids = stimulus_presentation_ids,
                                      stimulus_condition_id = stimulus_condition_id,
                                      session = session,
                                      **kwargs).index

example(get_stimulus_presentations(stimulus_name = 'static_gratings',
                                   orientation = AND(NOT('null'),
                                                     RANGE(30, 60, ub_strict=False))))

,stimulus_block,start_time,stop_time,orientation,x_position,color,phase,stimulus_name,frame,spatial_frequency,contrast,temporal_frequency,size,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
49434,8.0,5382.489060,5382.739264,60.0,null,null,0.5,static_gratings,null,0.32,0.8,null,"[250.0, 250.0]",null,0.250204,4789
49437,8.0,5383.239686,5383.489905,60.0,null,null,0.0,static_gratings,null,0.04,0.8,null,"[250.0, 250.0]",null,0.250219,4792
49442,8.0,5384.490745,5384.740946,60.0,null,null,0.25,static_gratings,null,0.16,0.8,null,"[250.0, 250.0]",null,0.250201,4797
49443,8.0,5384.740946,5384.991147,30.0,null,null,0.0,static_gratings,null,0.04,0.8,null,"[250.0, 250.0]",null,0.250201,4798
49447,8.0,5385.741767,5385.991973,30.0,null,null,0.5,static_gratings,null,0.16,0.8,null,"[250.0, 250.0]",null,0.250206,4802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70382,14.0,9133.639100,9133.889309,30.0,null,null,0.25,static_gratings,null,0.04,0.8,null,"[250.0, 250.0]",null,0.250209,4867
70383,14.0,9133.889309,9134.139517,60.0,null,null,0.75,static_gratings,null,0.04,0.8,null,"[250.0, 250.0]",null,0.250209,4886
70385,14.0,9134.389719,9134.639920,60.0,null,null,0.0,static_gratings,null,0.08,0.8,null,"[250.0, 250.0]",null,0.250201,4874


In [23]:
def get_presentationwise_spike_times(session: Session = CURRENT_SESSION, **kwargs):
    """Return a table of the spike times of the matching units and stimuli.

    All filters which `get_units` and `get_stimulus_presentations`
    accept are meaningful.

    """
    kwargs['__total__'] = False
    return session.presentationwise_spike_times(
        stimulus_presentation_ids = get_stimulus_presentation_ids(session = session, **kwargs),
        unit_ids = get_unit_ids(session = session, **kwargs)
    )

example(get_presentationwise_spike_times(stimulus_name = 'static_gratings',
                                         orientation = 0.0,
                                         ecephys_structure_acronym = 'VISam'))

,stimulus_presentation_id,unit_id,time_since_stimulus_presentation_onset
spike_time,,,
5385.245593,49445,950912018,0.004239
5385.246593,49445,950911932,0.005239
5385.260527,49445,950912249,0.019173
5385.263693,49445,950912283,0.022339
5385.266060,49445,950911932,0.024706
...,...,...,...
9130.866531,70370,950912226,0.229961
9130.867631,70370,950912164,0.231061
9130.875031,70370,950911932,0.238461


In [24]:
def get_conditionwise_spike_statistics(use_rates: Optional[bool] = False,
                                       session: Session = CURRENT_SESSION,
                                       **kwargs):
    """Return a table of the spike statistics for each stiulus condition.

    If `use_rates` is True, use firing rates, otherwise use spike counts.
    
    All filters which `get_units` and `get_stimulus_presentations`
    accept are meaningful.

    """
    kwargs['__total__'] = False
    return session.conditionwise_spike_statistics(
        stimulus_presentation_ids = get_stimulus_presentation_ids(session = session, **kwargs),
        unit_ids = get_unit_ids(session = session, **kwargs),
        use_rates = use_rates
    )

example(get_conditionwise_spike_statistics(ecephys_structure_acronym = 'VISam',
                                           stimulus_name = OR('static_gratings','drifting_gratings')))

,,spike_count,stimulus_presentation_count,spike_mean,spike_std,spike_sem
unit_id,stimulus_condition_id,,,,,
950911873,246,4,15,0.266667,0.798809,0.206252
950911880,246,353,15,23.533333,17.679151,4.564737
950911932,246,45,15,3.000000,2.725541,0.703732
950911986,246,6,15,0.400000,0.736788,0.190238
950912018,246,346,15,23.066667,12.109422,3.126639
...,...,...,...,...,...,...
950912601,4907,102,49,2.081633,1.987846,0.283978
950912646,4907,7,49,0.142857,0.456435,0.065205
950913000,4907,18,49,0.367347,0.698029,0.099718
